# Testing in Proteus

In [1]:
import proteus
import numpy as np
import numpy.testing as npt
from nose.tools import eq_ as eq
from nose.tools import ok_ as ok

Like any other large software project, Proteus relies on testing to ensure quality, aid development, and protect developers, both from each other and themselves.  There are a number of types of tests, but we'll focus on understanding the following:

* Unit tests
* Regression tests
* Documentation tests
* Smoke tests

Although these categories are useful for describing tests, another important characterization is the amount of time tests require to complete.  Although unit and smoke tests should also run quickly, this is not necessarily true for documentation or regression tests, which might take hours or even days to complete.

## Unit tests

Unit tests are probably the least understood of the testing types.  This is likely due to the popularity of so-called xUnit testing frameworks which greatly simplify the process of writing and using tests.  Although xUnit frameworks such as `nose` were designed to augment your ability to write unit tests, they can work equally well for other types of tests (though documentation tests frequently have their own frameworks).  

What distinguishes a unit test from other types of tests is that is designed to test the smallest functional components of the code in isolation from each other.  Advanced unit tests rely on techniques such as fixtures, which prepare a class or object for testing, and mocks, which are "fake" versions of another object you might otherwise interact with.  

You can run all the tests in `src/test/test_linear_algebra.py` by heading to the root of the Proteus directory and running:

In [7]:
!cd ~/proteus/proteus/tests/linalgebra_tests && \
py.test -v test_linear_algebra.py

============================= test session starts ==============================
platform linux -- Python 3.7.2, pytest-3.4.1, py-1.5.2, pluggy-0.6.0 -- /home/cekees/proteus/linux/bin/python
cachedir: ../../../.pytest_cache
rootdir: /home/cekees/proteus, inifile:
plugins: xonsh-0.9.6, xdist-1.22.2, forked-0.2, cov-2.5.1
collected 12 items                                                             

test_linear_algebra.py::test_vec_create PASSED                           [  8%]
test_linear_algebra.py::test_mat_create PASSED                           [ 16%]
test_linear_algebra.py::test_vec_scalar_math PASSED                      [ 25%]
test_linear_algebra.py::test_mat_vec_math PASSED                         [ 33%]
test_linear_algebra.py::test_superlu_mat PASSED                          [ 41%]
test_linear_algebra.py::test_norm_correctness PASSED                     [ 50%]
test_linear_algebra.py::test_norm_zero PASSED                            [ 58%]
test_linear_algebra.py::test_norm_hom

Here's a complete example of a unit test from the file `src/test/test_linear_algebra.py`

In [8]:
def test_vec_create():
    """test_vec_create

    Verifies that the proteus.LinearAlgebraTools.Vec constructor
    correctly creates one-dimensional arrays of the given length of
    type double precision and with entries set to zero for several
    trials.
    """
    from proteus.LinearAlgebraTools import Vec
    for n in [1, 10, 100, 1000]:
        x = Vec(n)
        # Vector of length n
        eq(x.size, n)
        # One-dimensional
        eq(x.shape, (n,))
        # Of type double-precision
        eq(x.dtype, np.double)
        # All entries are zero
        eq(np.count_nonzero(x), 0)
        # Verify assignment works
        x[:] = range(1, n+1)
        eq(np.count_nonzero(x), n)

In [9]:
test_vec_create()

The only important thing to note is that any function starting with the word `test` (and living in a module that starts with `test`) is automatically executed when we run the `nosetests` test runner from the command line.  The `eq` statements check that two expressions are equal.  In this case, verifying that we are creating NumPy arrays of the appropriate size, dimensions, and type for a few different dimensions.

## Regression tests

Regression tests are probably the most well-known type of test.  A regression test is used to indicate some known "success" in the history of Proteus.  Frequently, we use regression tests to ensure that externally validated results (say, from an actual wave tank or other physical model), do not change.  

The most difficult part in writing a regression test is automating validation.  As numerical scientists, it is easy to get caught up in the use of "eyeball norms".  It is crucial, however, that you do your best to distill the essence of your solution into its most important components (in lower dimensions, if possible).  Does the error converge at a specific rate?  Is the energy or mass of the system conserved?  Is there a characteristic frequency or easily computed gauge that would be sensitive to an incorrect solution or an easily made mistake?

## Documentation tests

Documentation tests combine regression or unit testing with documentation.  Here's a simple example:

In [13]:
!$(which python) -m proteus.LinearAlgebraTools -v

Trying:
    Mat(2,3)
Expecting:
    array([[ 0.,  0.,  0.],
          [ 0.,  0.,  0.]])
**********************************************************************
File "/home/cekees/proteus/proteus/LinearAlgebraTools.py", line 742, in __main__.Mat
Failed example:
    Mat(2,3)
Expected:
    array([[ 0.,  0.,  0.],
          [ 0.,  0.,  0.]])
Got:
    array([[0., 0., 0.],
           [0., 0., 0.]])
Trying:
    Vec(3)
Expecting:
    array([ 0.,  0.,  0.])
**********************************************************************
File "/home/cekees/proteus/proteus/LinearAlgebraTools.py", line 729, in __main__.Vec
Failed example:
    Vec(3)
Expected:
    array([ 0.,  0.,  0.])
Got:
    array([0., 0., 0.])
86 items had no tests:
    __main__
    __main__.InvOperatorShell
    __main__.InvOperatorShell.__init__
    __main__.InvOperatorShell._converged_trueRes
    __main__.InvOperatorShell._create_constant_nullspace
    __main__.InvOperatorShell._create_copy_vec
    __main__.InvOperatorShell._create_tmp

The code looks like this:

In [14]:
def Vec(n):
    """
    Build a vector of length n (using numpy)

    For example:
    >>> Vec(3)
    array([ 0.,  0.,  0.])
    """


The `>>>` is a sign to the doctest module that the documentation string contains an input, followed by an expected output.  You can use this to simultaneously document and test your code.

## Smoke Tests

Smoke tests are among the simplest tests to write.  A smoke test usually performs no validation or other verification, it's just a piece of code that you'd like to run to completion.  Many professional software engineers joke that scientific code is *one fix away from being completely broken*.  A smoke test ensures that the code is running, but it inspires little confidence that the code is actually running **correctly**.  